In [1]:
import pandas as pd
import numpy as np
import os
from typing import Callable # this is to define test_func functions dtype as function itself
import re # for regural expression
import warnings
warnings.filterwarnings('ignore')

In [2]:
print(os.getcwd())
os.chdir(r'd:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem')
print(os.getcwd())

d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem\notebook\model_building\DataForModel
d:\vscode_machineLearning\BEST_PROJECTS\mobileRecommenderSystem


In [3]:
mi = pd.read_csv(filepath_or_buffer=r'data\raw_data\mi_mobile_new_data.csv',   index_col=False)
samsung = pd.read_csv(filepath_or_buffer=r'data\raw_data\samsung_mobile_new_data.csv',   index_col=False)
apple = pd.read_csv(filepath_or_buffer=r'data\raw_data\apple_mobile_new_data.csv',   index_col=False)
onePlus = pd.read_csv(filepath_or_buffer=r'data\raw_data\one_plus_mobile_new_data.csv',   index_col=False)
realme = pd.read_csv(filepath_or_buffer=r'data\raw_data\realme_mobile_data.csv',   index_col=False)
vivo = pd.read_csv(filepath_or_buffer=r'data\raw_data\vivo_mobile_data.csv',   index_col=False)
oppo = pd.read_csv(filepath_or_buffer=r'data\raw_data\oppo_mobile_data.csv',   index_col=False)
motorola = pd.read_csv(filepath_or_buffer=r'data\raw_data\motorola_mobile_data.csv',   index_col=False)
techno = pd.read_csv(filepath_or_buffer=r'data\raw_data\techno_mobile_data.csv',   index_col=False)
infinix = pd.read_csv(filepath_or_buffer=r'data\raw_data\infinix_mobile_data.csv',   index_col=False)

In [4]:
infinix.rename({'camera':'os_processor','oS_Processor':'camera'},axis=1,inplace=True)
# techno.rename({'camera':'os_processor','oS_Processor':'camera'},axis=1,inplace=True)
motorola.rename({'camera':'os_processor','oS_Processor':'camera'},axis=1,inplace=True)
vivo.rename({'camera':'os_processor','oS_Processor':'camera'},axis=1,inplace=True)
realme.rename({'camera':'os_processor','oS_Processor':'camera'},axis=1,inplace=True)

dataframes_list = [mi, samsung, apple, onePlus, realme, vivo, oppo, motorola, techno, infinix]

In [5]:
for i in dataframes_list:
    i.drop_duplicates(inplace=True)

In [6]:
dataframes_dict = {
    'mi': mi,
    'samsung': samsung,
    'apple': apple,
    'onePlus': onePlus,
    'realme': realme,
    'vivo': vivo,
    'oppo': oppo,
    'motorola': motorola,
    'techno': techno,
    'infinix': infinix
}


## Shape

In [7]:
for i,j in dataframes_dict.items():
    print(f'{i} data shape :{j.shape}')
    

mi data shape :(517, 10)
samsung data shape :(566, 10)
apple data shape :(424, 10)
onePlus data shape :(188, 10)
realme data shape :(249, 10)
vivo data shape :(308, 10)
oppo data shape :(251, 10)
motorola data shape :(119, 10)
techno data shape :(151, 10)
infinix data shape :(165, 10)


## Handle Null values

In [8]:
for key,value in dataframes_dict.items():
    print(f'dataframe is {key} \n {sum(value.isnull().sum())}')
    print()

dataframe is mi 
 28

dataframe is samsung 
 81

dataframe is apple 
 322

dataframe is onePlus 
 0

dataframe is realme 
 35

dataframe is vivo 
 7

dataframe is oppo 
 0

dataframe is motorola 
 0

dataframe is techno 
 0

dataframe is infinix 
 2



In [9]:
mi.dropna(inplace=True)
realme.dropna(inplace=True)
vivo.dropna(inplace=True)
infinix.dropna(inplace=True)
samsung.dropna(inplace=True)

In [10]:
oppo[oppo['ratings'].isnull() ==True]

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery


In [11]:
# oppo['ratings'][46] = 4.3
# oppo['ratings'][47] = 4.3
# oppo['ratings'][167] = 4.5
# oppo['ratings'][191] = 4.1
# oppo['ratings'][214] = 4.3
# oppo['ratings'][215] = 4.3
# oppo['ratings'][239] = 4.4
# oppo['ratings'][263] = 4.4
# oppo['os_processor'][191] = 'Operating SystemAndroid Pie Android v9.0 (Pie)Processor CoreOcta CorePrimary Clock Speed0.1 MHz'

In [12]:
for key,value in dataframes_dict.items():
    print(f'dataframe is {key} \n {sum(value.isnull().sum())}')
    print()

dataframe is mi 
 0

dataframe is samsung 
 0

dataframe is apple 
 322

dataframe is onePlus 
 0

dataframe is realme 
 0

dataframe is vivo 
 0

dataframe is oppo 
 0

dataframe is motorola 
 0

dataframe is techno 
 0

dataframe is infinix 
 0



In [13]:
def random_values(col_name:str,df:pd.DataFrame,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

# Price

In [14]:
def clean_price_col(dataframe:pd.DataFrame):
    dataframe.reset_index(inplace=True)
    for Index,i in enumerate(dataframe['price']):
        i = str(i).replace('₹','')
        i = str(i).replace(',','')
        dataframe['price'][Index] = i

    dataframe['price'] = dataframe['price'].astype(int)
    dataframe['price'] = dataframe['price'].abs()  

    show = f'datatype -{dataframe["price"].dtype}'   
    return show

## storage_ram

In [15]:
def remove_internal(txt):
    return txt.replace('Internal','')

In [16]:
## Going to remove 'Storage32 GBRAM2 GBExpandable Storage128' words after this.....
def cut_remove_words(txt):
    split = txt.split()
    if 'GBExpandable' in split:
        string = split[:5]
        extract_GB_TB = string[-1]
        pattern = r'[TB]+|[GB]+'
        found = re.findall(pattern,extract_GB_TB)

        final_string = ' '.join(string[:4]) + ''.join(found)

        return final_string
    else:
        return ' '.join(split[:2])

In [17]:
def func_apply(func:Callable,col_name,df:pd.DataFrame,df_name=None):
    df[col_name] = df[col_name].apply(func)
    print(f'Successfully applied function in {df_name}')
    
    print(f'{random_values(col_name=col_name,df=df,total=6)}')
    

## remove Operating

In [18]:
def remove_operating(txt):
    return txt.replace('Operating','')

for i in dataframes_list:
    print(func_apply(func=remove_operating,col_name='os_processor',df=i,df_name=None))


Successfully applied function in None
345 :  SystemAndroid 12Processor TypeMediatek MT6769Z Helio G85Processor CoreOcta CorePrimary Clock Speed2 GHzSecondary Clock Speed1.8 GHz
212 :  SystemAndroid 12Processor TypeMediatek Dimensity 1080Processor CoreOcta CorePrimary Clock Speed2.6 GHzSecondary Clock Speed2 GHz Frequency2G GSM: B3/B5/B8, 3G WCDMA: B1/B5/B6/B8/B19, 4G LTE TDD: B34/B38/B39/B40/B41, 4G LTE FDD: B1/B3/B5/B8/B19, 5G SA: N1/N3/N5/N8/N28A/N38/N40/N41/N77/N78, 5G NSA: N38/N40/N41/N77/N78
87 :  SystemAndroid Q 11Processor CoreDual CorePrimary Clock Speed2.2 GHz
440 :  SystemAndroid Oreo 8.1Processor TypeQualcomm Snapdragon 660 AIEProcessor CoreOcta CorePrimary Clock Speed2.2 GHz FrequencyGSM: B2, B3, B5, B8, WCDMA: B1, B2, B5, B8, 4G LTE-TDD: B40, B41, 4G LTE-FDD: B1, B3, B5
94 :  SystemAndroid Q 11Processor CoreOcta CorePrimary Clock Speed3.2 GHz
156 :  SystemAndroid Nougat 7.1.2Processor TypeQualcomm Snapdragon 636Processor CoreOcta CorePrimary Clock Speed1.8 GHz FrequencyGSM

## ***os_processor***

In [19]:
def extract_processor(text):
    string = text.split()
    find = re.findall(r'Speed\d.*+',' '.join(string))
    find = ' '.join(find).split()
    pattern = r'.*Processor'
    match = re.findall(pattern, text)
    
    if match:
        return match[0]  + ' '+ ''.join(find[0])
    else:
        return text


In [20]:
for df_name ,df in dataframes_dict.items():
    if df_name == 'apple':
        continue
    else:
        print(func_apply(func=extract_processor,col_name='os_processor',df=df,df_name=df_name))  
    


Successfully applied function in mi
39 :  SystemAndroid Oreo 8.1Processor Type2.0 GHz Mediatek P22 OctacoreProcessor Speed2
203 :  SystemAndroid 11Processor TypeQualcomm Snapdragon 680Processor Speed2.4
270 :  SystemAndroid 12.5Processor Speed2
264 :  SystemAndroid 11Processor TypeMediatek Helio G95Processor Speed2.05
78 :  SystemAndroid 12Processor Speed2.2
145 :  SystemAndroid Pie 9Processor TypeQualcomm Snapdragon 675Processor Speed2
None
None
Successfully applied function in samsung
260 :  SystemAndroid Pie 9.0Processor TypeExynos 9611Processor Speed2.3
360 :  SystemAndroid 10Processor TypeExynos 850 Octa CoreProcessor Speed2
280 :  SystemAndroid 10Processor TypeQualcomm Snapdragon (SDM450-F01) Octa CoreProcessor Speed1.8
524 :  SystemAndroid Pie 9.0Processor Speed2
510 :  SystemAndroid 10Processor TypeExynos 990Processor Speed2.73
314 :  SystemAndroid 11Processor TypeMediaTek Dimensity 700 (MT6833V)Processor Speed2.2
None
None
Successfully applied function in onePlus
148 :  System

## camera

In [21]:
def camera_extract(row):
    row_splitted = row.split()
    pattern = r'\d+MP'
    words = ' '.join(row_splitted)
    find_cam = re.findall(pattern,words)
    return ' '.join(find_cam)


In [22]:
for df_name ,df in dataframes_dict.items():
    print(func_apply(func=camera_extract,col_name='camera',df=df,df_name=df_name))  
    


Successfully applied function in mi
209 : 50MP 2MP 50MP 2MP 5MP 5MP
72 : 8MP 5MP
362 : 64MP 8MP 2MP 2MP 64MP 8MP 2MP 2MP 64MP 13MP 13MP
229 : 50MP
295 : 48MP
291 : 50MP
None
None
Successfully applied function in samsung
16 : 50MP 5MP 2MP 50MP 5MP 2MP 8MP 8MP
194 : 64MP 12MP 5MP 5MP 64MP 12MP 5MP 5MP 32MP
420 : 64MP 12MP 5MP 5MP 32MP
209 : 50MP 12MP 10MP 50MP 12MP 10MP 10MP 10MP
268 : 50MP 50MP 2MP 2MP 50MP 2MP 2MP 13MP
456 : 13MP 2MP 13MP 8MP 5MP
None
None
Successfully applied function in apple
340 : 12MP 12MP 7MP
372 : 8MP 2MP
229 : 12MP 5MP
324 : 5MP 3MP
247 : 12MP 12MP 12MP 12MP 64MP 12MP 12MP
89 : 12MP 12MP 12MP 63MP 12MP 12MP
None
None
Successfully applied function in onePlus
23 : 48MP 8MP 32MP 8MP
187 : 50MP
171 : 48MP
64 : 50MP
94 : 50MP
32 : 50MP
None
None
Successfully applied function in realme
203 : 50MP 8MP 2MP 50MP 8MP 2MP 16MP 16MP
59 : 50MP 3MP 50MP 5MP 3MP 50MP 5MP 5MP
155 : 64MP 8MP 2MP 2MP 16MP
213 : 50MP 3MP 50MP 5MP 3MP 50MP 5MP 5MP
30 : 50MP 2MP 2MP 50MP 2MP 2MP 8MP

## network

In [23]:
def network(txt):
    pattern = r'\bNetwork Type(4G|5G|3G|2G)\b'
    search = re.search(pattern,txt)
    if search:
        find = re.findall(pattern,txt)
        return ''.join(find)
    else:
        pattern = r'\bNetwork Type(4G|5G|3G|2G)Supported'
        find = re.findall(pattern,txt)
        return ''.join(find)


In [24]:
for df_name ,df in dataframes_dict.items():
    print(func_apply(func=network,col_name='network',df=df,df_name=df_name))  
    


Successfully applied function in mi
296 : 4G
219 : 4G
77 : 5G
61 : 5G
447 : 3G
13 : 5G
None
None
Successfully applied function in samsung
243 : 5G
26 : 4G
453 : 4G
459 : 4G
252 : 5G
440 : 4G
None
None
Successfully applied function in apple
98 : 5G
180 : 5G
242 : 4G
222 : 3G
243 : 4G
269 : 3G
None
None
Successfully applied function in onePlus
78 : 5G
46 : 5G
92 : 5G
15 : 5G
182 : 5G
90 : 5G
None
None
Successfully applied function in realme
118 : 4G
150 : 5G
70 : 4G
117 : 5G
110 : 2G
60 : 4G
None
None
Successfully applied function in vivo
135 : 5G
296 : 4G
53 : 4G
61 : 5G
38 : 3G
270 : 4G
None
None
Successfully applied function in oppo
161 : 2G
19 : 4G
90 : 4G
93 : 4G
114 : 4G
1 : 4G
None
None
Successfully applied function in motorola
15 : 5G
97 : 5G
96 : 4G
75 : 5G
45 : 4G
12 : 2G
None
None
Successfully applied function in techno
63 : 4G
66 : 5G
16 : 4G
33 : 4G
113 : 4G
75 : 4G
None
None
Successfully applied function in infinix
103 : 4G
96 : 4G
9 : 2G
103 : 4G
97 : 4G
67 : 4G
None
None


## Battery

In [25]:
apple.fillna('notMentioned',inplace=True)

In [26]:
def extract_battery(txt):
    if txt == 'notMentioned':
        return 'notMentioned'
    else:
        capBattery = txt.split()[1]
        return capBattery 

for df_name, df in dataframes_dict.items():
    print(func_apply(func=extract_battery, col_name='battery', df=df, df_name=df_name))


Successfully applied function in mi
430 : Capacity3000
197 : Capacity5000
408 : Capacity4500
28 : Capacity6000
314 : Capacity5000
458 : Capacity4000
None
None
Successfully applied function in samsung
204 : Capacity4500
303 : Capacity3300
240 : Capacity4400
491 : Capacity5000
424 : Capacity4000
153 : Capacity5000
None
None
Successfully applied function in apple
343 : notMentioned
183 : notMentioned
145 : notMentioned
135 : notMentioned
326 : notMentioned
170 : notMentioned
None
None
Successfully applied function in onePlus
76 : Capacity4000
58 : Capacity3800
16 : Capacity5000
184 : Capacity4500
53 : Capacity3800
10 : Capacity4500
None
None
Successfully applied function in realme
82 : Capacity5000
56 : Capacity5000
12 : Capacity5000
154 : Capacity3500
179 : Capacity5000
176 : Capacity4300
None
None
Successfully applied function in vivo
39 : Capacity5000
142 : Capacity5000
215 : Capacity4300
224 : Capacity5000
264 : Capacity5000
271 : Capacity4000
None
None
Successfully applied function i

# Mi

In [27]:
mi

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"Redmi 9A Sport (Coral Green, 32 GB)",4.2,"₹8,990",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage32 GBRAM3 GBMemory Card Slot T...,SystemAndroid Q 10Processor Speed2,13MP,Display Size16.59 cm (6.53 inch)Resolution1600...,4G,Capacity5000
1,"Xiaomi 11Lite NE (Vinyl Black, 128 GB)",4.2,"₹21,430",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBExpandable Storag...,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 20MP,Display Size16.64 cm (6.55 inch)Resolution1080...,5G,Capacity4250
2,"Xiaomi 11Lite NE (Diamond Dazzle, 128 GB)",4.2,"₹20,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBExpandable Storag...,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 20MP,Display Size16.64 cm (6.55 inch)Resolution1080...,5G,Capacity4250
3,"Xiaomi 11Lite NE (Tuscany Coral, 128 GB)",4.2,"₹20,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBExpandable Storag...,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 20MP,Display Size16.64 cm (6.55 inch)Resolution1080...,5G,Capacity4250
4,"Mi 11 Lite (Jazz Blue, 128 GB)",4.2,"₹17,499",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GBRAM8 GBExpandable Storag...,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 64MP 8MP 5MP 16MP 16MP,Display Size16.64 cm (6.55 inch)Resolution2400...,4G,Capacity4250
...,...,...,...,...,...,...,...,...,...,...
546,"13 pro (Ceramic Black, 256 GB)",4.3,"₹79,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage256 GBRAM12 GBMemory Card Slot...,"SystemAndroid MIUI 14, Android 13.0Processor ...",50MP,Display Size17.09 cm (6.73 inch)Resolution3200...,5G,Capacity4820
547,"Mi A3 (Not just Blue, 64 GB)",4.2,"₹14,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid Pie 9Processor TypeQualcomm Sna...,48MP 8MP 2MP 48MP 8MP 2MP 32MP 32MP,Display Size15.46 cm (6.088 inch)Resolution156...,4G,Capacity4030
548,"Mi 10T Pro (Lunar Silver, 128 GB)",4.3,"₹26,899",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GBRAM8 GBCall Log MemoryYes,SystemAndroid 10Processor TypeQualcomm Snapdr...,108MP 13MP 5MP 108MP 13MP 5MP 108MP 13MP 5MP 20MP,Display Size16.94 cm (6.67 inch)Resolution2400...,5G,Capacity5000
549,"Redmi Y1 (Gold, 32 GB)",4.5,"₹8,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage32 GBRAM3 GBExpandable Storage...,SystemAndroid Nougat 7.1.2Processor TypeQualc...,13MP 16MP,Display Size13.97 cm (5.5 inch)Resolution1280 ...,3G,Capacity3080


### ***price***

In [28]:
clean_price_col(mi)

'datatype -int32'

### *****storage_ram*****

In [29]:
func_apply(func=remove_internal,col_name='storage_ram',df=mi)

Successfully applied function in None
225 :  Storage64 GBRAM4 GBMemory Card Slot TypeDedicated Slot
377 :  Storage64 GBRAM6 GBExpandable Storage512 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
264 :  Storage64 GBRAM6 GBExpandable Storage512 GBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
476 :  Storage64 GBRAM4 GBExpandable Storage256 GBMemory Card Slot TypeHybrid Slot
313 :  Storage128 GBRAM6 GBExpandable Storage512 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
126 :  Storage128 GBRAM6 GB
None


In [30]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=mi)

Successfully applied function in None
378 : Storage64 GBRAM6 GBExpandable Storage512GB
45 : Storage64 GBRAM6 GBExpandable Storage256GB
409 : Storage32 GBRAM3 GBExpandable Storage512GB
480 : Storage256 GBRAM8
302 : Storage128 GBRAM4 GBExpandable Storage512GB
112 : Storage128 GBRAM6 GBExpandable Storage512GB
None


# Samsung

In [31]:
samsung.dropna(inplace=True)

In [32]:
samsung

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"SAMSUNG Galaxy F13 (Sunrise Copper, 64 GB)",4.3,"₹9,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeExynos 850Proce...,50MP 5MP 2MP 50MP 5MP 2MP 8MP 8MP,Display Size16.76 cm (6.6 inch)Resolution2408 ...,4G,Capacity6000
1,"SAMSUNG Galaxy F13 (Waterfall Blue, 128 GB)",4.3,"₹10,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 12Processor TypeExynos 850Proce...,50MP 5MP 2MP 50MP 5MP 2MP 8MP 8MP,Display Size16.76 cm (6.6 inch)Resolution2408 ...,4G,Capacity6000
2,"SAMSUNG Galaxy F13 (Nightsky Green, 128 GB)",4.3,"₹10,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 12Processor TypeExynos 850Proce...,50MP 5MP 2MP 50MP 5MP 2MP 8MP 8MP,Display Size16.76 cm (6.6 inch)Resolution2408 ...,4G,Capacity6000
3,"SAMSUNG Galaxy F13 (Sunrise Copper, 128 GB)",4.3,"₹10,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 12Processor TypeExynos 850Proce...,50MP 5MP 2MP 50MP 5MP 2MP 8MP 8MP,Display Size16.76 cm (6.6 inch)Resolution2408 ...,4G,Capacity6000
4,"SAMSUNG Galaxy F23 5G (Aqua Blue, 128 GB)",4.3,"₹15,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBTotal Memory128 G...,SystemAndroid 12Processor TypeQualcomm Snapdr...,50MP 8MP 2MP 50MP 8MP 2MP 8MP 8MP,Display Size16.76 cm (6.6 inch)Resolution2408 ...,5G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
569,"SAMSUNG Galaxy M40 (Seawater Blue, 128 GB)",4.5,"₹21,490",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid Pie 9.0Processor Speed2,32MP,Display Size16.0 cm (6.3 inch)Resolution2340 x...,4G,Capacity3500
570,SAMSUNG Galaxy Note 20 Ultra 5G (Mystic Bronze...,4.3,"₹1,04,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage256 GBRAM12 GBExpandable Stora...,SystemAndroid 10Processor TypeExynos Octa Cor...,108MP 12MP 12MP 108MP 12MP 12MP 10MP 10MP,Display Size17.53 cm (6.9 inch)Resolution3088 ...,5G,Capacity4500
571,"SAMSUNG Galaxy A30s (Prism Crush Violet, 64 GB)",4.3,"₹16,199",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBSupported Memory C...,SystemAndroid Pie 9.0Processor TypeExynos 790...,25MP 5MP 8MP 25MP 5MP 8MP 16MP,Display Size16.26 cm (6.4 inch)Resolution1920 ...,3G,Capacity4000
572,"SAMSUNG Galaxy J4 Plus (Black, 32 GB)",4.4,"₹9,000",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage32 GBRAM2 GBExpandable Storage...,SystemAndroid Oreo 8.1Processor TypeQualcomm ...,13MP 5MP,Display Size15.24 cm (6 inch)Resolution720 x 1...,3G,Capacity3300


### ***price***

In [33]:
clean_price_col(samsung)

'datatype -int32'

### *****storage_ram*****

In [34]:
func_apply(func=remove_internal,col_name='storage_ram',df=samsung)

Successfully applied function in None
65 :  Storage64 GBRAM4 GBTotal Memory64 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated Slot
116 :  Storage256 GBRAM8 GBTotal Memory256 GB
222 :  Storage64 GBRAM4 GBTotal Memory64 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
176 :  Storage512 GBRAM12 GB
171 :  Storage128 GBRAM4 GBTotal Memory128 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated Slot
12 :  Storage64 GBRAM4 GBExpandable Storage1 TBSupported Memory Card TypeMIcroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
None


In [35]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=samsung)

Successfully applied function in None
502 : Storage32 GBRAM3 GBExpandable Storage512GB
218 : Storage128 GBRAM4 GBTotal Memory128GB
286 : Storage128 GBRAM4
146 : Storage128 GBRAM8 GBExpandable Storage1TB
474 : Storage64 GBRAM4 GBExpandable Storage512GB
79 : Storage64 GBRAM4
None


# apple

In [36]:
apple

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"APPLE iPhone 13 (Blue, 128 GB)",4.7,"₹61,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
1,"APPLE iPhone 13 (Midnight, 128 GB)",4.7,"₹61,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
2,"APPLE iPhone 13 (Starlight, 128 GB)",4.7,"₹61,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
3,"APPLE iPhone 13 (Pink, 128 GB)",4.7,"₹61,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
4,"APPLE iPhone 14 (Midnight, 128 GB)",4.6,"₹69,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GB,"SystemiOS 16Processor TypeA15 Bionic Chip, 6 ...",12MP 12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
...,...,...,...,...,...,...,...,...,...,...
419,"APPLE iPhone XR (Yellow, 128 GB)",4.6,"₹41,699",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 14.2Processor TypeA12 Bionic Chip,12MP 63MP 7MP,Display Size15.49 cm (6.1 inch)Resolution1792 ...,3G,notMentioned
420,"APPLE iPhone 11 Pro (Midnight Green, 256 GB)",4.6,"₹1,21,300",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage256 GB,SystemiOS 13Processor TypeA13 Bionic Chip,12MP 12MP 12MP 12MP 64MP 12MP 12MP,Display Size14.73 cm (5.8 inch)Resolution2436 ...,4G,notMentioned
421,"APPLE iPhone X (Silver, 64 GB)",4.6,"₹91,900",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage64 GB,SystemiOS 11Processor TypeA11 Bionic Chip wit...,12MP 12MP 63MP 7MP,Display Size14.73 cm (5.8 inch)Resolution2436 ...,3G,notMentioned
422,"APPLE iPhone X (Space Gray, 256 GB)",4.6,"₹1,06,900",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage256 GB,SystemiOS 11Processor TypeA11 Bionic Chip wit...,12MP 12MP 63MP 7MP,Display Size14.73 cm (5.8 inch)Resolution2436 ...,3G,notMentioned


### ***price***

In [37]:
clean_price_col(apple)

'datatype -int32'

In [38]:
apple

,index,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,0,"APPLE iPhone 13 (Blue, 128 GB)",4.7,61999,https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
1,1,"APPLE iPhone 13 (Midnight, 128 GB)",4.7,61999,https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
2,2,"APPLE iPhone 13 (Starlight, 128 GB)",4.7,61999,https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
3,3,"APPLE iPhone 13 (Pink, 128 GB)",4.7,61999,https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 15Processor TypeA15 Bionic ChipProc...,12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
4,4,"APPLE iPhone 14 (Midnight, 128 GB)",4.6,69999,https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GB,"SystemiOS 16Processor TypeA15 Bionic Chip, 6 ...",12MP 12MP 12MP 12MP 63MP 12MP 12MP,Display Size15.49 cm (6.1 inch)Resolution2532 ...,5G,notMentioned
...,...,...,...,...,...,...,...,...,...,...,...
419,419,"APPLE iPhone XR (Yellow, 128 GB)",4.6,41699,https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GB,SystemiOS 14.2Processor TypeA12 Bionic Chip,12MP 63MP 7MP,Display Size15.49 cm (6.1 inch)Resolution1792 ...,3G,notMentioned
420,420,"APPLE iPhone 11 Pro (Midnight Green, 256 GB)",4.6,121300,https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage256 GB,SystemiOS 13Processor TypeA13 Bionic Chip,12MP 12MP 12MP 12MP 64MP 12MP 12MP,Display Size14.73 cm (5.8 inch)Resolution2436 ...,4G,notMentioned
421,421,"APPLE iPhone X (Silver, 64 GB)",4.6,91900,https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage64 GB,SystemiOS 11Processor TypeA11 Bionic Chip wit...,12MP 12MP 63MP 7MP,Display Size14.73 cm (5.8 inch)Resolution2436 ...,3G,notMentioned
422,422,"APPLE iPhone X (Space Gray, 256 GB)",4.6,106900,https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage256 GB,SystemiOS 11Processor TypeA11 Bionic Chip wit...,12MP 12MP 63MP 7MP,Display Size14.73 cm (5.8 inch)Resolution2436 ...,3G,notMentioned


### *****storage_ram*****

In [39]:
func_apply(func=remove_internal,col_name='storage_ram',df=apple)

Successfully applied function in None
422 :  Storage256 GB
316 :  Storage64 GB
1 :  Storage128 GB
256 :  Storage16 GBCall Log MemoryYes
133 :  Storage1 TB
262 :  Storage256 GB
None


In [40]:
def remove_extra(txt):
    split = txt.split()
    word = split[0]
    if 'Storage1' in split: 
        replace = word.replace('Storage1','Storage1024')
        return replace
    else:
        return word
    
func_apply(func=remove_extra,col_name='storage_ram',df=apple)

Successfully applied function in None
186 : Storage128
337 : Storage64
301 : Storage128
40 : Storage64
385 : Storage16
164 : Storage256
None


### ***os_processor***

In [41]:
def extract_processor(text):
    pattern = r'.*Chip'
    match = re.findall(pattern, text)
    if match:
        return match[0]
    else:
        split = text.split()
        return ' '.join(split[:5])

func_apply(extract_processor,'os_processor',df=apple)

Successfully applied function in None
312 :  SystemiOS 12Processor TypeA12 Bionic Chip
124 :  SystemiOS 12Processor TypeA11 Bionic Chip
144 : SystemAndroid 13Processor TypeDimensity 7050Processor CoreOcta
201 :  SystemiOS 13Processor TypeA13 Bionic Chip
9 :  SystemiOS 14.2Processor TypeA13 Bionic Chip
273 :  SystemiOS 8Processor TypeA8 Chip
None


# onePlus

In [42]:
onePlus

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"OnePlus Nord CE 2 Lite 5G (Blue Tide, 128 GB)",4.4,"₹17,676",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid 10Processor Speed2.2,64MP,Display Size16.74 cm (6.59 inch)Resolution2412...,5G,Capacity5000
1,"OnePlus Nord CE 2 Lite 5G (Black Dusk, 128 GB)",4.4,"₹17,691",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid 13Processor Speed2.2,64MP,Display Size16.74 cm (6.59 inch)Resolution2412...,5G,Capacity5000
2,"OnePlus 11R 5G (Galactic Silver, 256 GB)",4.5,"₹44,690",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage256 GBRAM16 GB,SystemAndroid OxygenOS based on Android 13Pro...,50MP,Display Size17.02 cm (6.7 inch)Resolution2772x...,5G,Capacity5000
3,"OnePlus 11R 5G (Galactic Silver, 128 GB)",4.5,"₹39,734",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GB,SystemAndroid OxygenOS based on Android 13Pro...,50MP,Display Size17.02 cm (6.7 inch)Resolution2772x...,5G,Capacity5000
4,"OnePlus 11R 5G (Sonic Black, 256 GB)",4.5,"₹44,284",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage256 GBRAM16 GB,SystemAndroid Oxygen OxygenOS based on Androi...,50MP,Display Size17.02 cm (6.7 inch)Resolution2772x...,5G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
183,"OnePlus N20 SE (BLUE OASIS, 64 GB)",4.2,"₹15,674",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBMemory Card Slot T...,SystemAndroid Oxygen 12.1Processor Speed1600,50MP,Display Size16.66 cm (6.56 inch)Resolution1612...,4G,Capacity5000
184,"OnePlus 9 Pro 5G (Morning Mist, 128 GB)",3.8,"₹47,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GB,SystemAndroid 11Processor Speed2.4,48MP,Display Size17.02 cm (6.7 inch)Resolution3216 ...,5G,Capacity4500
185,"OnePlus 6 (Mirror Black, 128 GB)",4.2,"₹17,990",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBMemory Card Slot ...,SystemAndroid Oreo 8.1Processor Speed2.7,20MP 20MP 16MP 16MP,Display Size15.95 cm (6.28 inch)Resolution1080...,4G,Capacity3300
186,"OnePlus 6T McLaren Edition (Speed Orange, 256 GB)",4.0,"₹26,990",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage256 GBRAM10 GBMemory Card Slot...,SystemAndroid Oxygen 9Processor Speed2.8,20MP 16MP 20MP 16MP,Display Size16.28 cm (6.41 inch)Resolution2280...,4G,Capacity3300


### ***price***

In [43]:
clean_price_col(onePlus)

'datatype -int32'

### *****storage_ram*****

In [44]:
func_apply(func=remove_internal,col_name='storage_ram',df=onePlus)

Successfully applied function in None
141 :  Storage128 GBRAM6 GBMemory Card Slot TypeHybrid SlotCall Log MemoryYes
182 :  Storage128 GBRAM8 GB
9 :  Storage128 GBRAM8 GBMemory Card Slot TypeDedicated Slot
45 :  Storage128 GBRAM8 GB
131 :  Storage128 GBRAM8 GBMemory Card Slot TypeDedicated Slot
80 :  Storage128 GBRAM8 GB
None


In [45]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=onePlus)

Successfully applied function in None
39 : Storage256 GBRAM12
104 : Storage128 GBRAM8
137 : Storage256 GBRAM8
178 : Storage128 GBRAM8
10 : Storage128 GBRAM8
160 : Storage256 GBRAM12
None


# realme

In [46]:
realme

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"realme C33 2023 (Sandy Gold, 128 GB)",4.4,"₹10,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 12Processor TypeUnisoc T612Proc...,50MP 3MP 50MP 5MP 3MP 50MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
1,"realme C33 2023 (Aqua Blue, 128 GB)",4.4,"₹10,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 12Processor TypeUnisoc T612Proc...,50MP 3MP 50MP 5MP 3MP 50MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
2,"realme C33 2023 (Night Sea, 128 GB)",4.4,"₹10,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 12Processor TypeUnisoc T612Proc...,50MP 3MP 50MP 5MP 3MP 50MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
3,"realme C30 (Denim Black, 32 GB)",4.2,"₹5,999",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage32 GBRAM2 GBExpandable Storage...,SystemAndroid 11Processor TypeUnisoc T612Proc...,8MP 8MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
4,"realme C30 (Bamboo Green, 32 GB)",4.1,"₹6,499",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage32 GBRAM3 GBExpandable Storage...,SystemAndroid 11Processor TypeUnisoc T612Proc...,8MP 8MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
257,"realme GT 2 (Paper Green, 256 GB)",4.3,"₹38,999",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage256 GBRAM12 GBCall Log MemoryYes,SystemAndroid 12Processor TypeQualcomm Snapdr...,50MP 8MP 2MP 50MP 8MP 2MP 50MP 16MP 16MP,Display Size16.81 cm (6.62 inch)Resolution2400...,5G,Capacity5000
258,"realme GT 2 (Paper Green, 128 GB)",4.6,"₹34,999",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage128 GBRAM8 GBCall Log MemoryYes,SystemAndroid 12Processor TypeQualcomm Snapdr...,50MP 8MP 2MP 50MP 8MP 2MP 50MP 16MP 16MP,Display Size16.81 cm (6.62 inch)Resolution2400...,5G,Capacity5000
259,"realme C21Y (Cross Black, 32 GB)",4.4,"₹7,499",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage32 GBRAM3 GBExpandable Storage...,SystemAndroid 11Processor TypeUnisoc T610Proc...,13MP 2MP 2MP 13MP 2MP 2MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
260,"realme C33 (Sandy Gold, 64 GB)",4.4,"₹9,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeUnisoc T612Proc...,50MP 3MP 50MP 5MP 3MP 50MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000


### ***price***

In [47]:
clean_price_col(realme)

'datatype -int32'

### *****storage_ram*****

In [48]:
func_apply(func=remove_internal,col_name='storage_ram',df=realme)

Successfully applied function in None
111 :  Storage128 GBRAM8 GB
45 :  Storage32 GBRAM2 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated Slot
116 :  Storage32 GBRAM3 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
194 :  Storage64 GBRAM4 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated Slot
53 :  Storage32 GBRAM3 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
97 :  Storage64 GBRAM6 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
None


In [49]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=realme)

Successfully applied function in None
181 : Storage64 GBRAM4
31 : Storage128 GBRAM6 GBExpandable Storage1TB
3 : Storage32 GBRAM2 GBExpandable Storage1TB
71 : Storage256 GBRAM12
127 : Storage128 GBRAM4 GBExpandable Storage256GB
86 : Storage128 GBRAM4 GBExpandable Storage256GB
None


# vivo

In [50]:
vivo

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"vivo T2x 5G (Marine Blue, 128 GB)",4.4,"₹13,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid 13Processor TypeDimensity 6020P...,50MP 2MP 50MP 2MP 8MP 8MP,Display Size16.71 cm (6.58 inch)Resolution2408...,5G,Capacity5000
1,"vivo T2x 5G (Glimmer Black, 128 GB)",4.4,"₹13,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid 13Processor TypeDimensity 6020P...,50MP 2MP 50MP 2MP 8MP 8MP,Display Size16.71 cm (6.58 inch)Resolution2408...,5G,Capacity5000
2,"vivo T2x 5G (Aurora Gold, 128 GB)",4.4,"₹13,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid 13Processor TypeDimensity 6020P...,50MP 2MP 50MP 2MP 8MP 8MP,Display Size16.71 cm (6.58 inch)Resolution2408...,5G,Capacity5000
3,"vivo T2x 5G (Aurora Gold, 128 GB)",4.3,"₹15,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBMemory Card Slot ...,SystemAndroid 13Processor TypeDimensity 6020P...,50MP 2MP 50MP 2MP 8MP 8MP,Display Size16.71 cm (6.58 inch)Resolution2408...,5G,Capacity5000
4,"vivo T2x 5G (Glimmer Black, 128 GB)",4.3,"₹15,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBMemory Card Slot ...,SystemAndroid 13Processor TypeDimensity 6020P...,50MP 2MP 50MP 2MP 8MP 8MP,Display Size16.71 cm (6.58 inch)Resolution2408...,5G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
307,"vivo Y20 (Obsidian Black, 64 GB)",4.3,"₹16,990",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 10Processor TypeQualcomm Snapdr...,13MP 2MP 2MP 13MP 2MP 2MP 13MP 2MP 2MP 8MP 8MP,Display Size16.54 cm (6.51 inch)Resolution1600...,4G,Capacity5000
308,"vivo Y20 (Purist Blue, 64 GB)",4.3,"₹16,990",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 10Processor TypeQualcomm Snapdr...,13MP 2MP 2MP 13MP 2MP 2MP 13MP 2MP 2MP 8MP 8MP,Display Size16.54 cm (6.51 inch)Resolution1600...,4G,Capacity5000
309,"vivo S1 Pro (Dreamy White, 128 GB)",4.4,"₹20,990",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GBRAM8 GBExpandable Storag...,SystemAndroid Pie 9Processor TypeQualcomm Sna...,48MP 8MP 2MP 2MP 48MP 48MP 8MP 2MP 2MP 32MP 32MP,Display Size16.21 cm (6.38 inch)Resolution1080...,4G,Capacity4500
310,"vivo Y51A (Crystal Symphony, 128 GB)",4.3,"₹16,990",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GBRAM6 GBExpandable Storag...,SystemAndroid 11Processor TypeQualcomm Snapdr...,48MP 8MP 2MP 48MP 8MP 2MP 16MP 16MP,Display Size16.71 cm (6.58 inch)Resolution2408...,4G,Capacity5000


### ***price***

In [51]:
clean_price_col(vivo)

'datatype -int32'

### ***storage_ram***

In [52]:
func_apply(func=remove_internal,col_name='storage_ram',df=vivo)

Successfully applied function in None
129 :  Storage128 GBRAM8 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeHybrid SlotCall Log MemoryYes
150 :  Storage128 GBRAM8 GBMemory Card Slot TypeHybrid Slot
204 :  Storage128 GBRAM8 GBExpandable Storage1 TBSupported Memory Card TypemicroSDCall Log MemoryYes
208 :  Storage128 GBRAM4 GBExpandable Storage1 TBMemory Card Slot TypeHybrid SlotCall Log MemoryYes
68 :  Storage128 GBRAM6 GBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated Slot
159 :  Storage128 GBRAM8 GBExpandable Storage1 TBMemory Card Slot TypeDedicated Slot
None


In [53]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=vivo)

Successfully applied function in None
273 : Storage256 GBRAM8
45 : Storage128 GBRAM8
15 : Storage128 GBRAM4 GBExpandable Storage1TB
26 : Storage128 GBRAM4 GBExpandable Storage1TB
195 : Storage64 GBRAM6 GBExpandable Storage256GB
257 : Storage32 GBRAM3
None


# oppo

In [54]:
oppo

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"OPPO A17k (Blue, 64 GB)",4.5,"₹9,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM3 GBExpandable Storage...,SystemAndroid 12Processor TypeMediatek Helio ...,8MP 8MP 5MP 5MP,Display Size16.66 cm (6.56 inch)Resolution1612...,4G,Capacity5000
1,"OPPO A17 (Lake Blue, 64 GB)",4.4,"₹12,200",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBMemory Card Slot T...,SystemAndroid Q 12Processor TypeMediatek Heli...,50MP 3MP 50MP 3MP 3MP 5MP 5MP,Display Size16.66 cm (6.56 inch)Resolution1612...,4G,Capacity5000
2,"OPPO A17 (Midnight Black, 64 GB)",4.4,"₹12,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeMediatek Helio ...,50MP 3MP 3MP 50MP 3MP 3MP 5MP 5MP,Display Size16.66 cm (6.56 inch)Resolution1612...,4G,Capacity5000
3,"OPPO A78 5G (Glowing Black, 128 GB)",4.3,"₹18,142",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBMemory Card Slot ...,SystemAndroidProcessor Speed2,13MP,Display Size16.66 cm (6.56 inch)Resolution720 ...,5G,Capacity5000
4,"OPPO A17 (Lake Blue, 64 GB)",4.4,"₹12,499",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeMediatek Helio ...,50MP 3MP 3MP 50MP 3MP 3MP 5MP 5MP,Display Size16.66 cm (6.56 inch)Resolution1612...,4G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
248,"OPPO A5s (Gold, 64 GB)",4.3,"₹15,990",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid Oreo 8.1Processor TypeMediaTek ...,13MP 2MP 8MP,Display Size15.75 cm (6.2 inch)Resolution1520 ...,3G,Capacity4230
249,"OPPO Reno7 Pro 5G (Startrails Blue, 256 GB)",4.4,"₹34,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage256 GBRAM12 GBExpandable Stora...,SystemAndroid 11Processor TypeMediaTek Dimens...,50MP 8MP 2MP 50MP 8MP 2MP 32MP 32MP,Display Size16.51 cm (6.5 inch)Resolution2400 ...,5G,Capacity4500
250,"OPPO A5s (Red, 32 GB)",4.3,"₹13,990",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage32 GBRAM3 GBExpandable Storage...,SystemAndroid Oreo 8.1Processor TypeMTK MT676...,13MP 2MP 8MP,Display Size15.75 cm (6.2 inch)Resolution1520 ...,3G,Capacity4230
251,"OPPO A15S (Fancy White, 64 GB)",4.4,"₹11,309",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBMemory Card Slot T...,SystemAndroid Q 10.0Processor Speed2.3,13MP,Display Size16.56 cm (6.52 inch)Resolution720 ...,4G,Capacity4230


In [55]:
func_apply(func=remove_internal,col_name='storage_ram',df=oppo)

Successfully applied function in None
167 :  Storage32 GBRAM2 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
194 :  Storage64 GBRAM4 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
250 :  Storage64 GBRAM3 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
123 :  Storage128 GBRAM6 GBExpandable Storage1 TBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated Slot
187 :  Storage16 GBRAM3 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
35 :  Storage256 GBRAM8 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYesSMS Memory200
None


In [56]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=oppo)

Successfully applied function in None
71 : Storage64 GBRAM4 GBExpandable Storage256GB
139 : Storage64 GBRAM4 GBExpandable Storage256GB
97 : Storage32 GBRAM2 GBExpandable Storage256GB
202 : Storage128 GBRAM6
162 : Storage256 GBRAM12
12 : Storage64 GBRAM3 GBExpandable Storage1TB
None


# motorola

In [57]:
motorola

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"MOTOROLA g42 (Atlantic Green, 64 GB)",4.2,"₹9,999",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage64 GBRAM4 GBSupported Memory C...,SystemAndroid 12Processor TypeQualcomm Snapdr...,50MP 8MP 2MP 50MP 8MP 2MP 16MP 16MP,Display Size16.43 cm (6.47 inch)Resolution2400...,4G,Capacity5000
1,"MOTOROLA e13 (Cosmic Black, 64 GB)",4.0,"₹7,299",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 13 (Go Edition)Processor TypeUn...,13MP 13MP 5MP 5MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
2,"MOTOROLA G32 (Mineral Gray, 128 GB)",4.1,"₹11,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBSupported Memory ...,SystemAndroid 12Processor TypeQualcomm Snapdr...,50MP 8MP 2MP 50MP 8MP 2MP 16MP 16MP,Display Size16.51 cm (6.5 inch)Resolution2400 ...,4G,Capacity5000
3,"MOTOROLA e32 (Arctic Blue, 64 GB)",3.9,"₹7,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeMediatek Helio ...,50MP 2MP 50MP 2MP 8MP 8MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,4G,Capacity5000
4,"MOTOROLA g13 (Lavender Blue, 128 GB)",3.9,"₹9,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM4 GBExpandable Storag...,SystemAndroid 13Processor TypeHelio G85Proces...,50MP 2MP 2MP 50MP 2MP 2MP 8MP 8MP,Display Size16.51 cm (6.5 inch)Resolution1600 ...,2G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
115,"Moto E6s (Polished Graphite, 64 GB)",4.1,"₹7,499",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 11Processor TypeSnapdragon 662P...,13MP 2MP 13MP 2MP 8MP 8MP,Display Size15.49 cm (6.1 inch)Resolution1560 ...,4G,Capacity3000
116,"MOTOROLA G30 (Pastel Sky, 64 GB)",4.1,"₹10,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBSupported Memory C...,SystemAndroid 11Processor TypeSnapdragon 460P...,64MP 8MP 2MP 2MP 64MP 8MP 2MP 2MP 64MP 8MP 13M...,Display Size16.54 cm (6.51 inch)Resolution1600...,4G,Capacity5000
117,"MOTOROLA G10 Power (Breeze Blue, 64 GB)",4.2,"₹9,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBSupported Memory C...,SystemAndroid 11Processor TypeMediaTek Dimens...,48MP 8MP 2MP 2MP 48MP 8MP 2MP 2MP 8MP,Display Size16.54 cm (6.51 inch)Resolution1600...,4G,Capacity6000
118,"MOTOROLA Edge 20 Fusion 5G (Cyber Teal, 128 GB)",4.0,"₹18,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage128 GBRAM6 GBMemory Card Slot ...,SystemAndroid 12Processor TypeQualcomm Snapdr...,108MP 8MP 2MP 108MP 8MP 2MP 32MP 32MP,Display Size16.94 cm (6.67 inch)Resolution2400...,5G,Capacity5000


### ***price***

In [58]:
clean_price_col(motorola)

'datatype -int32'

### ***storage_ram***

In [59]:
func_apply(func=remove_internal,col_name='storage_ram',df=motorola)

Successfully applied function in None
65 :  Storage16 GBRAM2 GBExpandable Storage32 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
94 :  Storage64 GBRAM4 GBSupported Memory Card TypeMicroSDMemory Card Slot TypeHybrid Slot
106 :  Storage64 GBRAM4 GBMemory Card Slot TypeDedicated Slot
88 :  Storage0 MBExpandable Storage32 GBPhone Book Memory1600SMS Memory200
116 :  Storage64 GBRAM4 GBSupported Memory Card TypemicroSDMemory Card Slot TypeHybrid Slot
27 :  Storage64 GBRAM2 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated Slot
None


In [60]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=motorola)

Successfully applied function in None
63 : Storage128 GBRAM8
96 : Storage64 GBRAM4 GBExpandable Storage1TB
116 : Storage64 GBRAM4
61 : Storage128 GBRAM8
16 : Storage32 MBRAM4
14 : Storage128 GBRAM8
None


In [61]:
def random_values(col_name:str,df:pd.DataFrame,total=6):
    for i in range(0,total):
        index = np.random.randint(df.shape[0])
        print(index,':',df[col_name].iloc[index])        

random_values(col_name='storage_ram',df=motorola)

112 : Storage4 MBRAM4
24 : Storage256 GBRAM8
43 : Storage64 GBRAM4 GBExpandable Storage1TB
7 : Storage64 GBRAM4
61 : Storage128 GBRAM8
107 : Storage128 GBRAM6


# techno

In [62]:
techno

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"Tecno Pova 3 (Eco Black, 128 GB)",4.1,"₹13,998",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage128 GBRAM6 GBExpandable Storag...,SystemAndroid 12Processor TypeMediaTek Helio ...,50MP 2MP 2MP 8MP,Display Size17.53 cm (6.9 inch)Resolution1080 ...,4G,Capacity7000
1,"Tecno Pop 5 Pro (Deepsea Luster, 32 GB)",4.2,"₹6,960",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage32 GBRAM3 GBExpandable Storage...,SystemAndroid 11 (Go Edition)Processor TypeHe...,8MP 5MP,Display Size16.56 cm (6.52 inch)Resolution720 ...,4G,Capacity6000
2,"Tecno Pova 3 (Eco Black, 64 GB)",4.2,"₹10,849",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeMediaTek Helio ...,50MP 2MP 2MP 8MP,Display Size17.53 cm (6.9 inch)Resolution1080 ...,4G,Capacity7000
3,"Tecno Pova 3 (Tech Silver, 64 GB)",4.2,"₹10,978",https://rukminim2.flixcart.com/image/312/312/l...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeMediaTek Helio ...,50MP 2MP 2MP 8MP,Display Size17.53 cm (6.9 inch)Resolution1080 ...,4G,Capacity7000
4,"Tecno Spark Go 2023 (UYUNI BLUE, 32+3 GB)",4.1,"₹6,750",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage32+3 GBRAM3 GBMemory Card Slot...,SystemAndroid 12Primary Clock Speed2 GHz,13MP,Display Size16.66 cm (6.56 inch)Resolution720x...,4G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
146,"Tecno Camon 15 (SHOAL GOLD, 64 GB)",4.2,"₹12,499",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 10Processor TypeHelio P22 MT676...,48MP 2MP 2MP 48MP 2MP 2MP 16MP 16MP,Display Size16.64 cm (6.55 inch)Resolution720 ...,4G,Capacity5000
147,"Tecno Camon 19 Neo (Green, 6 GB)",4.1,"₹18,490",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage6 GBRAM128 GBMemory Card Slot ...,SystemAndroid Android 12Primary Clock Speed2 GHz,48MP,Display Size17.27 cm (6.8 inch)Resolution2460x...,2G,Capacity5000
148,"Tecno Spark 10 Pro (Starry Black, 256 GB)",4.1,"₹14,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage256 GBRAM8 GBExpandable Storag...,SystemAndroid 13Processor TypeMediatek MT6769...,50MP 32MP,Display Size17.22 cm (6.78 inch)Resolution2460...,4G,Capacity5000
149,"Tecno Spark 10 Pro (Pearl White, 256 GB)",4.1,"₹14,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage256 GBRAM8 GBExpandable Storag...,SystemAndroid 13Processor TypeMediatek MT6769...,50MP 32MP,Display Size17.22 cm (6.78 inch)Resolution2460...,4G,Capacity5000


### ***price***

In [63]:
techno['price'][95] = '15999'

In [64]:
clean_price_col(techno)

'datatype -int32'

In [65]:
techno[techno['price'] == 'nan']

,index,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery


In [66]:
random_values(col_name='storage_ram',df=techno)

6 : Internal Storage32+3 GBRAM3 GBMemory Card Slot TypeDedicated Slot
26 : Internal Storage64 GBRAM2 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
149 : Internal Storage256 GBRAM8 GBExpandable Storage1 TBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
85 : Internal Storage128 GBRAM8 GBExpandable Storage1 TBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
114 : Internal Storage128 GBRAM6 GBTotal Memory128 GBExpandable Storage128 GBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYesSMS MemoryYes
31 : Internal Storage32 GBRAM2 GBExpandable Storage256 GBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYes


### ***storage_ram***

In [67]:
func_apply(func=remove_internal,col_name='storage_ram',df=techno)

Successfully applied function in None
21 :  Storage64 GBRAM4 GBExpandable Storage512 GBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
32 :  Storage64 GBRAM4 GBExpandable Storage512 MBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
147 :  Storage6 GBRAM128 GBMemory Card Slot TypeDedicated Slot
34 :  Storage64 GBRAM3 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
67 :  Storage128 GBRAM8 GBExpandable Storage1 TBSupported Memory Card TypeMicro SDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
110 :  Storage64 GBRAM4 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
None


In [68]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=techno)

Successfully applied function in None
38 : Storage64 GBRAM3
10 : Storage64 GBRAM3
133 : Storage64 GBRAM4 GBExpandable Storage256GB
70 : Storage4 GBRAM4
61 : Storage128 GBRAM6
87 : Storage6 GBRAM128
None


# infinix

In [69]:
infinix

,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,"Infinix SMART 7 (Azure Blue, 64 GB)",4.3,"₹7,299",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeUnisoc Spreadtr...,13MP 13MP 5MP 5MP,Display Size16.76 cm (6.6 inch)Resolution1612 ...,4G,Capacity6000
1,"Infinix HOT 30i (Marigold, 128 GB)",4.3,"₹8,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBExpandable Storag...,SystemAndroid 12Processor TypeG37Processor Sp...,50MP 50MP 5MP 5MP,Display Size16.76 cm (6.6 inch)Resolution1612 ...,2G,Capacity5000
2,"Infinix HOT 30i (Marigold, 64 GB)",4.3,"₹7,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeG37Processor Sp...,50MP 50MP 5MP 5MP,Display Size16.76 cm (6.6 inch)Resolution1612 ...,2G,Capacity5000
3,"Infinix HOT 30i (Diamond White, 128 GB)",4.3,"₹8,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage128 GBRAM8 GBExpandable Storag...,SystemAndroid 12Processor TypeG37Processor Sp...,50MP 50MP 5MP 5MP,Display Size16.76 cm (6.6 inch)Resolution1612 ...,2G,Capacity5000
4,"Infinix HOT 30i (Diamond White, 64 GB)",4.3,"₹7,999",https://rukminim2.flixcart.com/image/312/312/x...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid 12Processor TypeG37Processor Sp...,50MP 50MP 5MP 5MP,Display Size16.76 cm (6.6 inch)Resolution1612 ...,2G,Capacity5000
...,...,...,...,...,...,...,...,...,...,...
187,"Infinix Hot S3X (Milan Black, 32 GB)",4.2,"₹10,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage32 GBRAM3 GBExpandable Storage...,SystemAndroid Oreo 8.1Processor TypeQualcomm ...,13MP 2MP 16MP,Display Size15.75 cm (6.2 inch)Resolution1500 ...,3G,Capacity4000
188,"Infinix Note 10 Pro (Nordic Secret, 256 GB)",4.2,"₹16,999",https://rukminim2.flixcart.com/image/312/312/k...,Internal Storage256 GBRAM8 GBExpandable Storag...,SystemAndroid 11Processor TypeMediaTek Helio ...,64MP 8MP 2MP 2MP 64MP 8MP 2MP 2MP 16MP 16MP,Display Size17.65 cm (6.95 inch)Resolution2460...,4G,Capacity5000
189,"Infinix Note 5 Stylus (Bordeaux Red, 64 GB)",3.9,"₹16,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid Oreo 8.1Processor TypeMediatek ...,16MP 16MP 16MP 16MP,Display Size15.06 cm (5.93 inch)Resolution1080...,3G,Capacity4000
190,"Infinix Note 5 Stylus (Charcoal Blue, 64 GB)",3.9,"₹16,999",https://rukminim2.flixcart.com/image/312/312/j...,Internal Storage64 GBRAM4 GBExpandable Storage...,SystemAndroid Oreo 8.1Processor TypeMediatek ...,16MP 16MP 16MP 16MP,Display Size15.06 cm (5.93 inch)Resolution1080...,3G,Capacity4000


## ***price***

In [70]:
clean_price_col(infinix)

'datatype -int32'

### ***storage_ram***

In [71]:
func_apply(func=remove_internal,col_name='storage_ram',df=infinix)

Successfully applied function in None
115 :  Storage64 GBRAM4 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
152 :  Storage32 GBRAM2 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
67 :  Storage128 GBRAM8 GBExpandable Storage256 GBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
4 :  Storage64 GBRAM4 GBExpandable Storage1 TBSupported Memory Card TypeMicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
155 :  Storage32 GBRAM3 GBExpandable Storage128 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
55 :  Storage32 GBRAM2 GBExpandable Storage256 GBSupported Memory Card TypemicroSDMemory Card Slot TypeDedicated SlotCall Log MemoryYes
None


In [72]:
func_apply(func=cut_remove_words,col_name='storage_ram',df=infinix)

Successfully applied function in None
109 : Storage16 GBRAM3 GBExpandable Storage128GB
159 : Storage256 GBRAM8 GBExpandable Storage256GB
127 : Storage64 GBRAM4 GBExpandable Storage256GB
12 : Storage64 GBRAM4 GBExpandable Storage256GB
129 : Storage128 GBRAM8 GBExpandable Storage256GB
135 : Storage128 GBRAM6 GBExpandable Storage256GB
None


In [73]:
random_values(col_name='storage_ram',df=infinix)

107 : Storage64 GBRAM4 GBExpandable Storage256GB
10 : Storage64 GBRAM2 GBExpandable Storage1TB
6 : Storage64 GBRAM4 GBExpandable Storage1TB
10 : Storage64 GBRAM2 GBExpandable Storage1TB
75 : Storage64 GBRAM4 GBExpandable Storage512GB
41 : Storage128 GBRAM8 GBExpandable Storage2TB


In [74]:
mi

,index,name,ratings,price,imgURL,storage_ram,os_processor,camera,display,network,battery
0,0,"Redmi 9A Sport (Coral Green, 32 GB)",4.2,8990,https://rukminim2.flixcart.com/image/312/312/x...,Storage32 GBRAM3,SystemAndroid Q 10Processor Speed2,13MP,Display Size16.59 cm (6.53 inch)Resolution1600...,4G,Capacity5000
1,1,"Xiaomi 11Lite NE (Vinyl Black, 128 GB)",4.2,21430,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 GBExpandable Storage1TB,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 20MP,Display Size16.64 cm (6.55 inch)Resolution1080...,5G,Capacity4250
2,2,"Xiaomi 11Lite NE (Diamond Dazzle, 128 GB)",4.2,20999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 GBExpandable Storage1TB,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 20MP,Display Size16.64 cm (6.55 inch)Resolution1080...,5G,Capacity4250
3,3,"Xiaomi 11Lite NE (Tuscany Coral, 128 GB)",4.2,20999,https://rukminim2.flixcart.com/image/312/312/x...,Storage128 GBRAM6 GBExpandable Storage1TB,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 20MP,Display Size16.64 cm (6.55 inch)Resolution1080...,5G,Capacity4250
4,4,"Mi 11 Lite (Jazz Blue, 128 GB)",4.2,17499,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM8 GBExpandable Storage512GB,SystemAndroid 11Processor TypeQualcomm Snapdr...,64MP 8MP 5MP 64MP 8MP 5MP 16MP 16MP,Display Size16.64 cm (6.55 inch)Resolution2400...,4G,Capacity4250
...,...,...,...,...,...,...,...,...,...,...,...
484,546,"13 pro (Ceramic Black, 256 GB)",4.3,79999,https://rukminim2.flixcart.com/image/312/312/x...,Storage256 GBRAM12,"SystemAndroid MIUI 14, Android 13.0Processor ...",50MP,Display Size17.09 cm (6.73 inch)Resolution3200...,5G,Capacity4820
485,547,"Mi A3 (Not just Blue, 64 GB)",4.2,14999,https://rukminim2.flixcart.com/image/312/312/k...,Storage64 GBRAM4 GBExpandable Storage256GB,SystemAndroid Pie 9Processor TypeQualcomm Sna...,48MP 8MP 2MP 48MP 8MP 2MP 32MP 32MP,Display Size15.46 cm (6.088 inch)Resolution156...,4G,Capacity4030
486,548,"Mi 10T Pro (Lunar Silver, 128 GB)",4.3,26899,https://rukminim2.flixcart.com/image/312/312/k...,Storage128 GBRAM8,SystemAndroid 10Processor TypeQualcomm Snapdr...,108MP 13MP 5MP 108MP 13MP 5MP 108MP 13MP 5MP 20MP,Display Size16.94 cm (6.67 inch)Resolution2400...,5G,Capacity5000
487,549,"Redmi Y1 (Gold, 32 GB)",4.5,8999,https://rukminim2.flixcart.com/image/312/312/j...,Storage32 GBRAM3 GBExpandable Storage128GB,SystemAndroid Nougat 7.1.2Processor TypeQualc...,13MP 16MP,Display Size13.97 cm (5.5 inch)Resolution1280 ...,3G,Capacity3080


In [75]:
for df_name ,df in dataframes_dict.items():
    try:
        df = df.drop(columns=['index'])
    except:
        print('notfound')
    # print(func_apply(func=camera_extract,col_name='camera',df=df,df_name=df_name))

notfound


In [77]:
mi.to_csv(path_or_buf=r'data\data_for_model\mi_mobile_new_data.csv',   index=False)
samsung.to_csv(path_or_buf=r'data\data_for_model\samsung_mobile_new_data.csv',   index=False)
apple.to_csv(path_or_buf=r'data\data_for_model\apple_mobile_new_data.csv',   index=False)
onePlus.to_csv(path_or_buf=r'data\data_for_model\one_plus_mobile_new_data.csv',   index=False)
realme.to_csv(path_or_buf=r'data\data_for_model\realme_mobile_data.csv',   index=False)
vivo.to_csv(path_or_buf=r'data\data_for_model\vivo_mobile_data.csv',   index=False)
oppo.to_csv(path_or_buf=r'data\data_for_model\oppo_mobile_data.csv',   index=False)
motorola.to_csv(path_or_buf=r'data\data_for_model\motorola_mobile_data.csv',   index=False)
techno.to_csv(path_or_buf=r'data\data_for_model\techno_mobile_data.csv',   index=False)
infinix.to_csv(path_or_buf=r'data\data_for_model\infinix_mobile_data.csv',   index=False)

> #                                                                   **END**